In [43]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import datetime
import os.path
import sys
import backtrader as bt
import numpy as np
from scipy.stats import norm
import yfinance as yf
import pandas as pd

In [44]:
# Black-Scholes Call Delta
def bs_delta(curr_price, strike_price, T, r, sigma):
    # T: time to expiration (% of year)
    # r: continuously compounded risk-free interest rate
    # sigma: volatility
    d1 = (np.log(curr_price / strike_price) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)
    
class DeltaHedgeStrategy(bt.Strategy):
    params = dict(
        options = 1,
        strike_price = 25,
        sigma = 0.2,
        r = 0.01,
        expiry_days = 30
    )

    def log(self, txt, dt=None):
        ''' Logging function'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.days = 0
        self.size = self.p.options * 100 # assuming each option includes 100 shares
        
    def hedge_fun(self, curr_price):
        T = (self.p.expiry_days - self.days) / 252 # convert days left to percent of year
        delta = bs_delta(curr_price, self.p.strike_price, T, self.p.r, self.p.sigma) # find delta
        target_shares = int(delta * self.size) # find updated target share amount
        
        if target_shares != self.position.size:
            self.log(f'Adjustment: Current={self.position.size}, Target={target_shares}, Delta={delta}')
            self.order = self.order_target_size(target=target_shares)
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0]) # log closing price
        curr_price = self.dataclose[0]
        
        if self.days == 0: # sell 1 call option at t=0
            self.log(f'Sell 1 Call Option: Strike={self.p.strike_price}')
            self.hedge_fun(curr_price)
            
        elif self.days < self.p.expiry_days: # if t>0 and option has not expired, update delta and hedge
            self.hedge_fun(curr_price)

        elif self.position:
            self.log('Option expired, closing hedge')
            self.close()
        
        self.days += 1

In [45]:
if __name__ == '__main__':
    
    data = bt.feeds.YahooFinanceCSVData(
        dataname='oracle.csv',
        fromdate=datetime.datetime(2000, 1, 1),
        todate=datetime.datetime(2000, 2, 28),
        reverse=False)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(DeltaHedgeStrategy)
    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    cerebro.run()

    print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')

2000-01-03, Close, 26.27
2000-01-03, Sell 1 Call Option: Strike=25
2000-01-03, Adjustment: Current=0, Target=77, Delta=0.7792991583160003
2000-01-04, Close, 23.95
2000-01-04, Adjustment: Current=77, Target=28, Delta=0.28044051542705584
2000-01-05, Close, 22.68
2000-01-05, Adjustment: Current=28, Target=7, Delta=0.0791392348470768
2000-01-06, Close, 21.35
2000-01-06, Adjustment: Current=7, Target=0, Delta=0.00909558756551954
2000-01-07, Close, 22.99
2000-01-07, Adjustment: Current=0, Target=10, Delta=0.10446266843598856
2000-01-10, Close, 25.74
2000-01-10, Adjustment: Current=10, Target=69, Delta=0.6950831812853634
2000-01-11, Close, 24.99
2000-01-11, Adjustment: Current=69, Target=51, Delta=0.5158772866836691
2000-01-12, Close, 23.49
2000-01-12, Adjustment: Current=51, Target=16, Delta=0.16211919620919812
2000-01-13, Close, 23.36
2000-01-13, Adjustment: Current=16, Target=13, Delta=0.13482522851460882
2000-01-14, Close, 23.75
2000-01-14, Adjustment: Current=13, Target=19, Delta=0.19902